# Higher-order models of paths

**September 08 2020**  
*Vincenzo Perri*

So far, we have focused on network models, but the real **purpose of `pathpy` is to fit and analyse higher-order models for paths in complex networks**. We will delve in higher order models in a moment in a moment, first we are going to focus on paths, the basic building block for the construction of Higher Order Networks. 

Like nodes and edges, in pathpy also paths are represented through objects using the `path` class.
A path is a collection of consecutive edge objects and can be created as follows.

In [1]:
import pathpy as pp

In [2]:
node_a = pp.Node("a")
node_b = pp.Node("b")
node_c = pp.Node("c")
node_d = pp.Node("d")


edge_ab = pp.Edge(node_a,node_b)
edge_bc = pp.Edge(node_b,node_c)
edge_cd = pp.Edge(node_c,node_d)

In [3]:
path = pp.Path(edge_ab,edge_bc,edge_cd, uid = "abcd")


Notice that it is the node object and not the node uid that counts for the creation of consecutive edges

In [4]:
e_ab = pp.Edge(pp.Node("a"),pp.Node("b"))
e_bc = pp.Edge(pp.Node("b"),pp.Node("c"))
e_cd = pp.Edge(pp.Node("c"),pp.Node("d"))

path_abcd = pp.Path(e_ab,e_bc,e_cd)

[09-07 19:44:13: ERROR] Path object needs consecutive edges!


AttributeError: 

The reason behind this becomes clear knowing that

In [5]:
pp.Node("a") == pp.Node("a")

False

Similarly to a network, we can get the number of nodes and edges in path using the `number_of_nodes` and `number_of_edges` functions, or we could just compute `len` of `Path.nodes` and `Path.edges`:

In [6]:
print('Path has {0} nodes and {1} edges'.format(path.number_of_nodes(), path.number_of_edges()))
print('Number of nodes: {0}'.format(len(path.nodes)))
print('Number of edges: {0}'.format(len(path.edges)))

Path has 4 nodes and 3 edges
Number of nodes: 4
Number of edges: 3


Paths can be grouped in a `PathCollection` object, this represents observed transitions (sequences) over the nodes of a network.

In [7]:
paths = pp.PathCollection()
paths.add(path, frequency = 20)

We can inspect the Path objects contained in a path collection

In [8]:
paths.values()

dict_values([Path abcd])

In [9]:
for path in paths.values():
    print(path.uid, ":" , path.nodes)

abcd : [Node a, Node b, Node c, Node d]


We can access the path objects in the path collection through their uid

In [10]:
paths["abcd"]

Path abcd

The method outlined above was rather cumbersome. Indeed Path collections can be created more conveniently by directly passing paths as sequences of node uids to the path collection object

In [11]:
paths = pp.PathCollection()
paths.add('a', 'c', 'd', uid='acd')
paths.add('b', 'c', 'e', uid='bce')

In [12]:
for path in paths.values():
    print(path.uid , path.nodes)

acd [Node a, Node c, Node d]
bce [Node b, Node c, Node e]


Or alternatively, can be imported using the `read_path_collection` function

In [13]:
toy_paths = pp.io.csv.read_pathcollection('data/toy_paths.ngram', frequency=True)

In [14]:
toy_paths.values()

dict_values([<pathpy.core.path.Path object at 0x000001D1EEC4A0D0>, <pathpy.core.path.Path object at 0x000001D1EEC61EB0>])

In [15]:
for path in paths.values():
    print(path.uid , path.nodes)

acd [Node a, Node c, Node d]
bce [Node b, Node c, Node e]


A network can easily be created from a path collection using the class function of `Network`

In [16]:
network = pp.Network.from_paths(toy_paths)
network

But pathpy was not created to reduce paths into networks. Sets of paths carry more information than sets of edges and pathpy's purpose is not to lose that information using *higher-order* network models.

From a higher-order network analytic point of view, standard graphs or networks are **first-order probabilistic generative models** for paths in complex networks. As we have seen in 1.2, they can be viewed as **maximum entropy models that consider first-order dyad statistics** (i.e. edge frequencies), while ignoring higher-order dependencies in real-world path, sequence, or time series data.

In the following works, we have studied measures for higher-order correlations in such data and we generalised network models to higher-order models with arbitrary order:

- R Pfitzner, I Scholtes, A Garas, CJ Tessone, F Schweitzer: **Betweenness Preference: Quantifying Correlations in the Topological Dynamics of Temporal Networks**, In Physical Review Letters, May 2013, [arXiv 1208.0588](http://arxiv.org/abs/1208.0588)

- I Scholtes, N Wider, R Pfitzner, A Garas, CJ Tessone, F Schweitzer: **Causality-driven slow-down and speed-up of diffusion in non-Markovian temporal networks**, In Nature Communications, September 2014, [arXiv 1307.4030](http://arxiv.org/abs/1307.4030)

- I Scholtes, N Wider, A Garas: **Higher-Order Aggregate Networks in the Analysis of Temporal Networks: Path structures and centralities**, In The European Physical Journal B, March 2016, [arXiv 1508.06467](http://arxiv.org/abs/1508.06467) 

- I Scholtes: **When is a Network a Network? Multi-Order Graphical Model Selection in Pathways and Temporal Networks**, In KDD'17, February 2017, [arXiv 1702.05499](https://arxiv.org/abs/1702.05499)

A broader overview of the research on higher-order models for complex systems is available the following article:

- R Lambiotte, M Rosvall, I Scholtes: **From Networks to Optimal Higher-Order Models of Complex Systems**, Nature Physics, March 2019,  [arXiv 1806.05977](https://arxiv.org/abs/1806.05977)

The data analysis and modelling framework outlined in these works builds on a generalisation of standard, first-order networks to $k$-dimensional De Bruijn graph models for paths in complex networks.

The class `HigherOrderNetwork` allows us to generate such higher-order network models of paths. In the documentation, we find that the constructor takes a parameter `paths`, i.e. the statistics of the observed paths that we want to model. With the parameter `k` we specify the order $k$ of the higher-order model that we want to fit. To understand this better, let us do this for our toy example.



A higher order network can either be created directly 

In [17]:
hon_1 = pp.HigherOrderNetwork().from_paths(toy_paths, order = 1)
print(hon_1)

Uid:			0x1d1eec7a430
Type:			HigherOrderNetwork
Order:			1
Number of nodes:	5
Number of edges:	4
Sub path statistics
 path  |          frequencies          |    unique    
length |    obs       pos       tot    |  obs    pos  
------ | --------- --------- --------- | ------ ------
     0 |         0        60        60 |      0      5
     1 |         0        40        40 |      0      4
------ | --------- --------- --------- | ------ ------
     1 |         0       100       100 |      0      9
obs ... observed paths (in the network)
pos ... possible paths (but not observed)
tot ... total number of paths


Or fitted:

In [18]:
hon_1 = pp.HigherOrderNetwork()
hon_1.fit(toy_paths,order = 1)
print(hon_1)

Uid:			0x1d1eec8ea30
Type:			HigherOrderNetwork
Order:			1
Number of nodes:	5
Number of edges:	4
Sub path statistics
 path  |          frequencies          |    unique    
length |    obs       pos       tot    |  obs    pos  
------ | --------- --------- --------- | ------ ------
     0 |         0        60        60 |      0      5
     1 |         0        40        40 |      0      4
------ | --------- --------- --------- | ------ ------
     1 |         0       100       100 |      0      9
obs ... observed paths (in the network)
pos ... possible paths (but not observed)
tot ... total number of paths


This generates a first-order model of our paths, with five nodes $a,b,c,d$ and $e$, and four links $(a,c), (b,c), (c,d), (c,e)$. It is identicaly to the `Network` instance that we have previously created using `Network.from_paths`.  We can store edge and node attributes and visualise higher order networks by exactly the same methods.

In [19]:
hon_1

We can see this network as a **first-order model** for paths where **edges are paths of length one**. That is, in a model with order $k=1$ edge weights capture the statistics of paths of length $k=1$.

We can generalise this idea to **k-th-order models** for paths, where **nodes are paths of length $k-1$** while **edge weights capture the statistics of paths of length $k$**. We can generate such a $k$-th order model by performing a line graph transformation on a model with order $k-1$. That is, edges in the model of order $k-1$ become nodes in the model with order $k$. We then draw edges between higher-order nodes whenever there is a possible path of length $k$ in the underlying network. The result is a $k$-dimensional De Bruijn graph model for paths. Let us try this in our example.

In [20]:
hon_2 = pp.HigherOrderNetwork()
hon_2.fit(toy_paths, order = 2)
hon_2

In [21]:
for edge in hon_2.edges:
    print(edge,edge['frequency'])

(a, c, d) 10.0
(b, c, e) 10.0


Each of the four **edges** in the first-order model is now represented by a **node** in the second-order model. We further have two directed edges $(a-c, c-d)$ and $(b-c,c-e)$ that represent the two paths of length two that occur in our data.

This is important because it captures to what extent the paths that we observe in our data deviate from what we would expect based on the (first-order) network topology of the system. Considering such a first-order model, all four paths $a \rightarrow c \rightarrow d, a \rightarrow c \rightarrow e, b \rightarrow c \rightarrow d$ and $b \rightarrow c \rightarrow e$ of length two are possible. If edges were statistically independent we would expect those four paths to occur with the same frequency.

Another way to express this independence assumption is to consider Markov chain models for the sequences of nodes traversed by a path. In this view, independently occurring edges translate to a memoryless first-order Markov process for the node sequence. In our example, we expect paths $a \rightarrow c \rightarrow d$ and $a \rightarrow c \rightarrow e$ to occur with the same probability, i.e. the next nodes $d$ or $e$ on a path through $c$ are independent from the previous node $a$, their probabilities only depending on the relative frequency of edges $(c,d)$ vs. $(c,e)$. In our toy example, we have a total of 20 observed paths of length two, so we expect each of the path to occur 5 times on average.

`pathpy` can actually generate this **null-model** for paths within the space of possible second-order models. This allows us to  compare how the observed path statistics deviate from a (Markovian) expectation.

In [22]:
hon_2_null = pp.NullModel(order = 2)
hon_2_null.fit(toy_paths)
hon_2_null

In [23]:
for edge in hon_2_null.edges:
    print(edge,edge['frequency'])

(a, c, d) 5.0
(a, c, e) 5.0
(b, c, d) 5.0
(b, c, e) 5.0


We can easily find out which of the paths of length two occur more or less often than expected under the null model. We can just subtract the adjacency matrices of the two instances `hon_2` and `hon_2_null`.


In [24]:
A_2 = hon_2.adjacency_matrix(weight='frequency')
n_2 = list(hon_2.nodes)

A_2_null = hon_2_null.adjacency_matrix(weight='frequency')
n_2_null = list(hon_2_null.nodes)

for e in hon_2_null.edges:
    v1,w1 = (n_2.index(hon_2.nodes[n.nodes]) for n in e.nodes)
    v2,w2 = (n_2_null.index(n) for n in e.nodes)
    print(e,A_2[v1,w1]-A_2_null[v2,w2])

(a, c, d) 5.0
(a, c, e) -5.0
(b, c, d) -5.0
(b, c, e) 5.0


This analysis confirms that the paths `b-c-e` and `a-c-d` occur five more times than we would expect at random, while the other two paths do occur five times fewer than expected (i.e. not at all). This deviation from our expectation changes the causal topology of the system, i.e. who can influence whom. In a network model we implicitly assume that paths are transitive, i.e. since `a` is connected to `c` and `c` is connected to `e` we assume that there is a path by which `a` can influence `e` via node `c`. The second-order model of our toy example reveals that this transitivity assumption is misleading, highlighting higher-order dependencies in our data that result in the fact that neither `a` can influence `e`, nor `b` can influence `d`.

### Ranking nodes in higher-order networks

Higher-order models capture deviations from the transitivity assumption that we often implicitly make when we apply standard graph mining or network analysis techniques. An important class of methods that rely on this assumption are centrality measures, which are often used to rank nodes in networked systems.

Let us consider the betweenness value of a node $v$, which (in its unnormalized variant) captures for how many pairs of nodes $x,y$ the shortest path from $x$ to $y$ passes through $v$. This measure is important, because is teaches us something about the role of nodes in information flow. It further captures to what extent removing a node will affect the shortest paths between other nodes. Let us try this in our example:

In [25]:
pp.algorithms.betweenness_centrality(hon_1)['c']

4.0

This is easy to understand, as there are four pairs `b-e`, `b-d`, `a-e`, and `a-d` for which the shortest path in the topology passes through node `c`. However, this notion of *importance* of node `c` is based on the assumption that paths are transitive and Markovian, which is not the case in our observed paths. `pathpy` actually allows us to calculate the betweenness of node `c` in the observed paths `paths`.

In [26]:
pp.algorithms.betweenness_centrality(paths)['c']

2.0

Not surprisingly, the betweenness of node $c$ in the actual paths is two, as $c$ only connects two pairs of nodes are connected via a (shortest) path. This change in centrality is captured in the higher-order model and `pathpy` allows us to directly generalize centrality measures to higher orders, simply by calling the functions on the class `HigherOrderNetwork`.

In [27]:
pp.algorithms.betweenness_centrality(hon_2)['c']

2.0

We see that in this example (since we **only** have second-order dependencies) the second-order model accurately captures the betweenness of node $c$. Let us contrast this to the second-order null model.

In [28]:
pp.algorithms.betweenness_centrality(hon_2_null)['c']

4.0

This is what we expect, since this null model is simply a second-order representation of the first-order model, in which paths are transitive and memoryless.

# Temporal Network Analysis and Visualisation in `pathpy`



### Introducing the `TemporalNetwork` class

We have considered the `Paths` class, which is useful if you have direct access to path statistics in your time series data. This includes clickstreams of users in information networks, origin-destination statistics in transportation networks, flight ticket sequences, or other **collections of short, ordered sequences**.

In this unit, we expand this view towards temporal networks, i.e. high-resolution time series network data, where edges carry fine-grained time stamps. Considering technical, social, and biological systems that can be modelled as dynamic networks, such data cover a broad class of complex systems that can be studied with higher-order network models.

`pathpy` provides special support for the analysis of temporal networks data via its `TemporalNetwork` class. It is suitable for data that captures time-stamped edges $(v, w, t)$ instantaneously occurring at discrete time stamps $t$. Let us start by creating an empty instance of this class.

In [29]:
t = pp.TemporalNetwork()
print(t)

Uid:			0x1d1eeb0adc0
Type:			TemporalNetwork
Directed:		True
Multi-Edges:		False
Number of unique nodes:	0
Number of unique edges:	0
Number of temp edges:	0
Observation periode:	0 - 0


The `TemporalNetwork` instance `t` stores two key information: a list of nodes `t.nodes` and a collection `t.tedges` of time-stamped edges  $(v, w, t)$. Let us add some time-stamped edges to this instance.

In [30]:
t.add_edge('a', 'b', t = 1)
t.add_edge('b', 'a', t = 3)
t.add_edge('b', 'c', t =3)
t.add_edge('d', 'c', t =4)
t.add_edge('c', 'd', t =5)
t.add_edge('c', 'b', t =6)
print(t)

Uid:			0x1d1eeb0adc0
Type:			TemporalNetwork
Directed:		True
Multi-Edges:		False
Number of unique nodes:	4
Number of unique edges:	6
Number of temp edges:	6
Observation periode:	1 - 7.0


We get basic summary statistics, such as the number of time-stamped interactions, the minimum and maximum timestamp, the duration of the observation, the number of different timestamps, as well as the average, minimum, and maximum time difference between consecutive time-stamped edges.

Just like other `pathpy` objects, we can directly embed interactive visualisations of a `TemporalNetwork` in-line in `jupyter`. Let us try this with our first example instance `t`.

In [31]:
# t

Again, this generates an embedded **interactive** visualisation, i.e. you can pan and zoom, or drag nodes. The controls in the top part of the visualisation allow you to stop, start or restart the simulation. 

We can easily save such interactive visualisations as stand-alone HTML5 files, which can be distributed via the Web.

In [32]:
pp.plot(t, 'visualisations/temporal_network.html')

In [33]:
pp.plot(t, 'visualisations/temporal_network.tex')

### Calculating path statistics in temporal networks

In the previous sections, **we modelled, analysed and visualised path statistics using higher-order network models**. But how can we  apply this higher-order analytics framework to time-stamped network data?

The key idea is that the ordering and timing in which time-stamped edges occur in a `TemporalNetwork` gives rise to so-called **causal or time-respecting paths**. In a nutshell, for two time-stamped edges $(a, b, t)$ and $(b, c, t')$ to contribute to a causal path $a \rightarrow b \rightarrow c$ it must hold that $t < t'$. If we swap timestamps such that the edge $(b, c)$ occurs **before** (a,b), no causal path $a \rightarrow b \rightarrow c$ exists.

So we observe that the chronological order of time-stamped edges crucially influences causal paths, i.e. which nodes can possibly influence each other in time-stamped edge sequences. Moreover, we often want to limit the **maximum time difference between consecutive edges** that contribute to a causal path. For data on dynamic social interactions that spans several years, it does not make sense to consider all chronologically ordered edges as possible causal paths for the propagation of information. After all, humans have limited memory and we should thus consider interactions that occur far apart in time as independent.

We can formally add this condition by setting a maximum time difference $\delta$ for the path calculation. That is, we only consider two edges $(a, b, t)$ and $(b, c, t')$ to contribute to a causal path $a \rightarrow b \rightarrow c$ if $ 0 < t' - t \leq \delta$.

With this definition at hand, and by setting our maximum time difference $\delta$, we can now **calculate causal path statistics in time-stamped network data**. `pathpy` provides powerful algorithms to calculate (or estimate) causal path statistics based on a `TemporalNetwork` instance. Let us try this in our toy example.

In [34]:
paths = t.to_paths()
paths.values()

dict_values([<pathpy.core.path.Path object at 0x000001D1EECC7550>, <pathpy.core.path.Path object at 0x000001D1EECC75E0>, <pathpy.core.path.Path object at 0x000001D1EECC7D90>, <pathpy.core.path.Path object at 0x000001D1EECC7E50>, <pathpy.core.path.Path object at 0x000001D1EECC7490>])

We see we got a set of paths, see how this changes if we change the delta:

In [35]:
paths = t.to_paths(delta=2)
paths.values()

dict_values([<pathpy.core.path.Path object at 0x000001D1EED23970>, <pathpy.core.path.Path object at 0x000001D1EED23DC0>, <pathpy.core.path.Path object at 0x000001D1EED23B20>, <pathpy.core.path.Path object at 0x000001D1EED23040>])



### Higher-order clustering and visualisation of causal structures in temporal networks

An interesting application of higher-order models concerns the visualisation of time-aggregated representations of temporal networks. We demonstrate this in a temporal network synthetically generated to exhibit a specific pattern in the ordering of time-stamped edges. Let us load an example and visualise the first time stamps of it.

In [36]:
import pathpy as pp
from utils import *

In [37]:
t = pp.io.csv.read_temporal_network('data/temporal_clusters.csv')

In [38]:
# t

What you probably cannot see in this visualisation is that this temporal network exhibits **clusters in the causal topology**, i.e. there are more **causal paths** between certain clusters of nodes than we would expect if links were independent (and paths were Markovian). We can get a vague idea of this by simulating a random walk in the temporal network, following the trajectory of a walker as it is moved through the nodes by dynamic edges.

In [39]:
import pickle
walk = pickle.load(open('temporal_walk.pkl', 'rb'))

In [40]:
# plot_walk(t,walk)

Still, the pattern is difficult to see as the layout of nodes does not reflect the causal structures in the temporal network. With `pathpy` we can easily get around this problem. We can generate a **time-aware layout** that considers the causal topology of the temporal network that results from higher-order dependencies between time-stamped edges. The basic approach of calculating higher-order layouts for temporal networks is described in this paper:

- V Perri, I Scholtes: [HOTVis: Higher-Order Time-Aware Visualisation of Dynamic Graphs](https://arxiv.org/abs/1908.05976), Graph Drawing 2020

We load previously computed causal paths a higher order network at the second order:

In [41]:
paths = pp.io.csv.read_pathcollection('data/temporal_misc.ngram', frequency=True)

In [42]:
hon_2 = pp.HigherOrderNetwork.from_paths(paths, order=2)

sub-path calculation: 100%|███████████████████████████████████████████████████████| 5999/5999 [00:24<00:00, 244.83it/s]


In [65]:
# plot_hon(hon_2)

We see that nodes automatically position themselves in groups that correspond to the natural cluster structure in the causal topology. In the visualisation above, we have additionally colored nodes based to their (synthetically generated) ground-truth clusters.

The cluster pattern in the layout actually disappears if we calculate a third-order layout.

In [44]:
hon_3 = pp.HigherOrderNetwork.from_paths(paths, oder=3)

sub-path calculation: 100%|███████████████████████████████████████████████████████| 5999/5999 [00:10<00:00, 557.90it/s]


In [45]:
# plot_hon(hon_3)

The reason for this is simple: This example exhibits correlations at correlation length two (i.e. for paths of length two), but there are no correlations at correlation length three. This highlights two important issues, that we will address in a moment:

1. We need a method to determine the optimal order of the higher-order models that we use to analyse (or visualise) a given data set.
2. As real systems are likely to exhibit multiple correlation length simultaneously, we need models that can combine multiple higher-order models.

# Multi-order models of paths

So far, we studied higher-order network models for path data with a fixed, given order $k$. We have seen that such higher-order models can yield better predictions compared to standard network models. But we also came across new questions: For temporal clusters, we could improve the detection of clusters moving from a first- to a second-order model. However, we did not see a further improvement for the third-order model. So in practice, how can we decide **at which order we should model a given system**? This points to a more general question as we can also imagine systems for which path statistics do not deviate significantly from the transitive, Markovian assumption made by a first-order model. So we **need methods to decide when higher-order models are actually needed**.

Moreover, a higher-order model with order $k$ can only capture higher-order dependencies at a single fixed correlation length $k$. But we may encounter data that exhibit multiple correlation lengths at once. How can we **combine models with multiple higher orders into a multi-order model**?

In this unit, we take a statistical inference and machine learning perspective to answer these questions. To show how the method works, we again start with a maximally simple toy example:


We recreate our toy example setting the frequency of each path to two:

In [46]:
import pathpy as pp

toy_paths = pp.PathCollection()
toy_paths.add('a', 'c', 'd', frequency=2)
toy_paths.add('b', 'c', 'e', frequency=2)

As mentioned before, in this example we only observe two of the four paths of length two that would be possible in the null model. Hence, this is an example for path statistics that exhibit correlations that warrant a second-order model. 

But how can we decide this in a statistically sound way? We can take a statistical inference perspective on the problem. More specifically, we will consider our higher-order networks as probabilistic generative models for paths in a given network topology. For this, let us use the weighted first-order network model to construct a transition matrix of a Markov chain model for paths in a network. We simply use the relative frequencies of edges to proportionally scale the probabilities of edge transitions in the model.

In [47]:
hon1 = pp.HigherOrderNetwork.from_paths(toy_paths, order = 1)
hon1.plot()
print(hon1.transition_matrix())

  (0, 1)	1.0
  (1, 4)	0.5
  (1, 2)	0.5
  (3, 1)	1.0


This transition matrix can be viewed as a first-order Markov chain model for paths in the underlying network topology. This probabilistic view allows us to calculate a likelihood of the first-order model, given the paths that we have observed. With `pathpy`, we can directly calculate the likelihood of a higher-order model, given a `Paths` instance.

In [48]:
print(hon1.likelihood(toy_paths, log=False))

0.0625


This result is particularly easy to understand for our toy example. Each path of length two corresponds to two transitions in the transition matrix of our Markov chain model. For each of the four paths of length two in `toy_paths`, the first transition is deterministic because nodes $a$ and $b$ only point to node $c$. However, based on the network topology, for the second step we have a choice between nodes $d$ and $e$. Considering that we see as many transitions through edge $(c,d)$ as we see through edge $(c,e)$, in a first-order model we have no reason to prefer one over the other, so each is assigned probability $0.5$.

Hence, for each of the four observed paths we obtain a likelihood of $1 \cdot 0.5 = 0.5$, which yields a total likelihood for four (independent) observations of $0.5^{4} = 0.0625$.

Let us compare this to the likelihood of a second-order model for our paths.

In [49]:
hon2 = pp.HigherOrderNetwork.from_paths(toy_paths, order = 2)
print(hon2.transition_matrix())
hon2.plot()
hon2.likelihood(toy_paths, log=False)

  (0, 1)	1.0
  (2, 3)	1.0


1.0

Here, the likelihood assumes its maximal value of $1.0$, simply because all transitions in the second-order model are deterministic, i.e. we simply multiply $1 \cdot 1$ four times. 

Let us now have a look at the *second-order null model*, which is actually a first-order model represented in the second-order space. So we should expect the same likelihood as the first-order model.

In [50]:
hon2_null = pp.NullModel.from_paths(toy_paths, order=2)
hon2_null.plot()
print(hon2_null.transition_matrix())
hon2_null.likelihood(toy_paths, log=False)

  (0, 2)	0.5
  (0, 1)	0.5
  (3, 2)	0.5
  (3, 1)	0.5


0.0625

### Model selection for higher-order network models

Clearly, the second-order null should have the same likelihood as the first-order model. This also shows a way to test hypotheses about the presence of higher-order correlations in paths. We can use a likelihood ratio test to compare the likelihood of the null hypothesis (i.e. a second-order representation of the first-order model) with the likelihood of an alternative hypothesis (the *fitted* second-order model).

But what do we learn from the fact that the likelihood of a model increases as we increase the order of the model. By itself, not much. Higher-order models are more complex than first-order models, i.e. while fitting their transition matrix we actually fit more parameters to the data. We can thus expect that such a more complex model better explains our (path) data. 

We should remind ourselves about Occam's razor, which states that we should favor models that make fewer assumptions. That is, in the comparison of the model likelihoods we should account for the additional complexity (or degrees of freedom) of a higher-order model over the null hypothesis.

A nice feature of our framework is that the null model and the alternative model are actually **nested**, i.e. the null model is one particular point in the parameter space of the (more general) higher-order model. Thanks to this property, we can apply [Wilk's theorem](https://en.wikipedia.org/wiki/Likelihood-ratio_test#Distribution:_Wilks’_theorem) to derive an analytical expression for the $p$-value of the null hypothesis that second-order correlations are absent (i.e. that a first-order model is sufficient to explain the observed paths), compared to the alternative hypothesis that a second-order model is needed. You can find the full mathematical details of this hypothesis testing approach in the following KDD'17 paper:

I Scholtes: [When is a Network a Network? Multi-Order Graphical Model Selection in Pathways and Temporal Networks](http://dl.acm.org/citation.cfm?id=3098145), In KDD'17 - Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, Halifax, Nova Scotia, Canada, August 13-17, 2017

Let us apply this to test the hypothesis that there are **significant** second-order dependencies in our toy example. This test consists of three basic steps: 

1. We calculate the difference $d$ between the parameters (or degrees of freedom) of a second- and a first-order model.
2. We calculate the test statistic $x = -2 \cdot(\log(\text{hon_1.likelihood}) - \log(\text{hon_2.likelihood}))$ for the likelihood ratio test.
3. We calculate a p-value as $1-cdf(x, d)$, where $cdf$ is the cumulative distribution function of a chi-square distribution.

In [51]:
network = pp.Network.from_paths(toy_paths)

In [52]:
hon2_null.degrees_of_freedom()

2

In [53]:
hon1_null = pp.NullModel.from_paths(toy_paths, order = 1)
hon2_null = pp.NullModel.from_paths(toy_paths, order = 2)

In [54]:
hon1_null.degrees_of_freedom()

1

In [55]:
hon2_null.degrees_of_freedom()

2

In [56]:
from scipy.stats import chi2

d = hon2_null.degrees_of_freedom() - hon1_null.degrees_of_freedom()
x = - 2 * (hon1.likelihood(toy_paths, log=True) - hon2.likelihood(toy_paths, log=True))
p = 1 - chi2.cdf(x, d)

print('The p-value of the null hypothesis (first-order model) is {0}'.format(p))

The p-value of the null hypothesis (first-order model) is 0.018531677751199016


The $p$-value of the null hypothesis that we can explain the four observed paths based on the weighted network topology alone is (borderline) 0.019. This is intuitive, as we have only observed four paths, which is hardly enough to robustly reject a first-order network model. Let us see what happens if we observe those same paths more often.


In [57]:
toy_paths = pp.PathCollection()
toy_paths.add('a', 'c', 'd', frequency=20)
toy_paths.add('b', 'c', 'e', frequency=20)

hon1 = pp.HigherOrderNetwork.from_paths(toy_paths, order = 1)
hon2 = pp.HigherOrderNetwork.from_paths(toy_paths, order = 2)

In [58]:
x = - 2 * (hon1.likelihood(toy_paths, log=True) - hon2.likelihood(toy_paths, log=True))
p = 1 - chi2.cdf(x, d)

print('The p-value of the null hypothesis (first-order model) is {0}'.format(p))

The p-value of the null hypothesis (first-order model) is 9.581224702515101e-14


So, if we were to observe each of the two paths four time, we would reject the null hypothesis because it is very unlikely to not observe two of the four possible paths a single time in eight observations. If we were to further increase the number of observations of the two paths the p-value decreases.

Unfortunately, the toy example above is too simple in multiple ways: First, it only contains paths of exactly length two, thus justifying a second-order model. But real data are more complex, as we have observations of paths at multiple lengths simultaneously. Such data are likely to exhibit multiple correlation lengths at the same time.

Even more importantly, in real data the model selection will unfortunately not work as described above. In fact, I have cheated because we cannot - in general - directly compare likelihoods of models with different order. The following example highlights this problem:

We create an empty `Paths` instance and add the following path:

`('a','b','c','d','e','c','b','a','c','d','e','c','e','d','c','a')`

Then we generate a first-order model, as well as a second- and fifth-order **null** model for the data and compare the likelihoods between the three models.

In [59]:
import pathpy as pp

In [60]:
path = pp.PathCollection()
path.add('a','b','c','d','e','c','b','a','c','d','e','c','e','d','c','a',frequency=1)

hon1 = pp.HigherOrderNetwork.from_paths(path,order=1)
hon2_null = pp.NullModel.from_paths(path,order=2)
hon5_null = pp.NullModel.from_paths(path,order=5)

print(hon1.likelihood(path, log=False))
print(hon2_null.likelihood(path, log=False))
print(hon5_null.likelihood(path, log=False))

1.755829903978052e-06
3.511659807956104e-06
2.633744855967078e-05


This is strange! Shouldn't the likelihoods of these three models be identical? They are not, and this is a major issue when we have data that consists of large numbers of short paths: in terms of the number of transitions that enter the likelihood calculation, a model of order $k$ discards the first $k$ nodes on each path. That is, a second-order model can only account for all but the first edge traversals on the path. This means that - in the general case - we actually compare likelihoods computed for different sample spaces, which is not valid.

### Multi-order representation learning

To fix the issues above, we need a probabilistic generative model that can deal with extensive collections of (short) paths in a network. The key idea is to combine multiple higher-order network models into a single multi-layered, multi-order model. To calculate the likelihood of such a model, we can use all layers, thus avoiding the problem that we discard prefixes of paths. For each path, we start the calculation at a layer of order zero, which considers the relative probabilities of nodes. We then use this model layer to calculate the probability to observe the first node on a path. For the next transition to step two, we then use a first-order model. The next transition is calculated in the second-order model and so on until we have reached the maximum order of our multi-order model. At this point, we can transitively calculate the likelihood based on the remaining transitions of the path.

The method is described in all details in the following paper:

I Scholtes: [When is a Network a Network? Multi-Order Graphical Model Selection in Pathways and Temporal Networks](http://dl.acm.org/citation.cfm?id=3098145), In KDD'17 - Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, Halifax, Nova Scotia, Canada, August 13-17, 2017

But let us go to practice. `pathpy` can directly generate, visualise, and analyse multi-order network models. Let us try this in our example.

In [61]:
mog = pp.MultiOrderModel.from_paths(toy_paths, max_order=2)
print(mog)

Multi-order model
- General --------------------------------------------
layer  |        network        |         DoF         
order  |   nodes      edges    |   paths      ngrams  
     0 |          5          0 |          4          4
     1 |          5          4 |          1         20
     2 |          4          2 |          2        100


We can now use the `likelihood` function of the class `MultiOrderModel` to repeat our likelihood ratio test. Rather than generating multiple `MultiOrderModel` instances for different hypotheses, we can directly calculate likelihoods based on different model layers within the same `MultiOrderModel` instance.

However, rather than performing the likelihood test ourselves, we can actually simply call the method `MultiOrderModel.estimate`. It will return the maximum order among all of its layers for which the likelihood ratio test rejects the null hypothesis.

In [62]:
mog.predict()

2

We now test whether this approach to **learn the optimal representation of path data** actually works. For this, let us generate path statistics that are in line with what we expect based on a first-order network model, and check whether the order estimation gives the right result.

In [63]:
random = pp.PathCollection()
random.add('a','c','d',frequency=5)
random.add('a','c','e',frequency=5)
random.add('b','c','d',frequency=5)
random.add('b','c','e',frequency=5)

mom_2 = pp.MultiOrderModel.from_paths(random,max_order=2)
mom_2.predict()

1